In [2]:
import pandas as pd

In [14]:
# Add Data into app
premier_league_url = "https://fbref.com/en/comps/13/2022-2023/2022-2023-Ligue-1-Stats"

# Only want to keep the first dataframe and the columns indicated below
prem_league_df = pd.read_html(premier_league_url)[0]
prem_columns = ["Rk", "Squad", "MP", "W", "L", "GF", "GA", "GD", "Pts", "Top Team Scorer", "Notes"]
prem_df = prem_league_df[prem_columns]
prem_df.to_csv('./data/France2022-2023.csv', index=False)

In [16]:
df = pd.read_csv('./data/ChampionsLeagueData.csv').dropna()[['Season', 'Champion', 'Runner-Up', 'Top Scorer']]
df.head()

,Season,Champion,Runner-Up,Top Scorer
1,2023-2024,Real Madrid,Dortmund,"Harry Kane, Kylian Mbappé - 8"
2,2022-2023,Manchester City,Internazionale,Erling Haaland - 12
3,2021-2022,Real Madrid,Liverpool,Karim Benzema - 15
4,2020-2021,Chelsea,Manchester City,Erling Haaland - 10
5,2019-2020,Bayern Munich,Paris Saint-Germain,Robert Lewandowski - 15


In [11]:
# Add champions league data to the analysis
champions_league_url = "https://fbref.com/en/comps/8/history/Champions-League-Seasons"

# Only want to keep the first dataframe and the columns indicated below
champions_league_df = pd.read_html(champions_league_url)[0]
champions_columns = ["Season", "Champion", "Runner-Up", "Top Scorer"]
champions_df = champions_league_df[champions_columns]
champions_df.to_csv('./data/ChampionsLeagueData.csv')